In [2]:
import pandas as pd

app_data = pd.read_csv('data/application_train.csv')
app_test_data = pd.read_csv('data/application_test.csv')

previous_transaction_summary = pd.read_csv('data/summary/previous_apps_summary.csv')
bureau_summary = pd.read_csv('data/summary/bureau_summary.csv')
pos_summary = pd.read_csv('data/summary/pos_cash_balance_summary.csv')
installment_payments_summary = pd.read_csv('data/summary/installment_payments_summary.csv')

app_data = app_data.merge(previous_transaction_summary, how='left', on='SK_ID_CURR')
app_data = app_data.merge(bureau_summary, how='left', on='SK_ID_CURR')
app_data = app_data.merge(pos_summary, how='left', on='SK_ID_CURR')
app_data = app_data.merge(installment_payments_summary, how='left', on='SK_ID_CURR')

app_test_data = app_test_data.merge(previous_transaction_summary, how='left', on='SK_ID_CURR')
app_test_data = app_test_data.merge(bureau_summary, how='left', on='SK_ID_CURR')
app_test_data = app_test_data.merge(pos_summary, how='left', on='SK_ID_CURR')
app_test_data = app_test_data.merge(installment_payments_summary, how='left', on='SK_ID_CURR')

In [3]:
app_data['CREDIT_PROPORTION_INCOME'] = app_data['AMT_CREDIT'] / app_data['AMT_INCOME_TOTAL']
app_test_data['CREDIT_PROPORTION_INCOME'] = app_test_data['AMT_CREDIT'] / app_test_data['AMT_INCOME_TOTAL']

app_data['ANNUITY_INCOME_PERCENT'] = app_data['AMT_ANNUITY'] / app_data['AMT_INCOME_TOTAL']
app_test_data['ANNUITY_INCOME_PERCENT'] = app_test_data['AMT_ANNUITY'] / app_test_data['AMT_INCOME_TOTAL']

app_data['CREDIT_TERM'] = app_data['AMT_ANNUITY'] / app_data['AMT_CREDIT']
app_test_data['CREDIT_TERM'] = app_test_data['AMT_ANNUITY'] / app_test_data['AMT_CREDIT']

app_data['DAYS_EMPLOYED_PERCENT'] = app_data['DAYS_EMPLOYED'] / app_data['DAYS_BIRTH']
app_test_data['DAYS_EMPLOYED_PERCENT'] = app_test_data['DAYS_EMPLOYED'] / app_test_data['DAYS_BIRTH']

In [4]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

target = 'TARGET'
classification_features = [col for col in app_data.columns if col not in (target,'SK_ID_CURR')]

app_data['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

encoder = LabelEncoder()

for feature in classification_features:
    feature_data = app_data[feature]
    # Looked into data, no numeric columns that represent categorical data, so safe to assume only object cols are categorical
    if feature_data.dtype == 'object' and len(feature_data.unique()) <= 2:
        encoder.fit(app_data[feature])
        app_data[feature] = encoder.transform(app_data[feature])
        app_test_data[feature] = encoder.transform(app_test_data[feature])

target_col = app_data['TARGET']        

app_data = pd.get_dummies(app_data)
app_test_data = pd.get_dummies(app_test_data)

app_data, app_test_data = app_data.align(app_test_data, join='inner', axis=1)
app_data['TARGET'] = target_col

In [5]:
ignore_cols = []

In [7]:
from xgboost import XGBClassifier

X_train, y_train = app_data.drop(['TARGET', 'SK_ID_CURR'] + ignore_cols, axis=1).values, app_data['TARGET'].values

# lr_model = LogisticRegression(C=0.0001)
# lr_model.fit(X_train, y_train)
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [8]:
app_test_data_X = app_test_data.drop(['SK_ID_CURR'] + ignore_cols, axis=1).values

predictions = xgb_model.predict_proba(app_test_data_X)[:, 1]

In [9]:
submission = pd.DataFrame()
submission['SK_ID_CURR'] = app_test_data['SK_ID_CURR']
submission['TARGET'] = predictions

In [6]:
from tpot import TPOTClassifier

X_train, y_train = app_data.drop(['TARGET', 'SK_ID_CURR'] + ignore_cols, axis=1).values, app_data['TARGET'].values

tpot = TPOTClassifier(generations=10, population_size=50, verbosity=2, n_jobs=-1)
tpot.fit(X_train, y_train)

/Users/dan/Envs/home-credit-default-risk/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Imputing missing values in feature set


ImportError: IntProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html